In [1]:
from langchain_groq import ChatGroq
from secret_key import groq_api_key

In [3]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"  # EXACT match
)

response = llm.invoke("Price of the mercedes glc 300 coupe in canda in cad ...")
print(response.content)

The price of the Mercedes-Benz GLC 300 Coupe in Canada can vary depending on the trim level, options, and features. Here are the current prices for the GLC 300 Coupe in Canada, in CAD:

**Starting Price:**

* GLC 300 Coupe: around $63,400 CAD (before taxes and fees)

**Trim Levels and Prices:**

* GLC 300 Coupe 4MATIC: $63,400 CAD
* GLC 300 Coupe 4MATIC AMG Line: $68,400 CAD
* GLC 300 Coupe 4MATIC AMG Line with Premium Package: $72,400 CAD
* GLC 300 Coupe 4MATIC AMG Line with Premium and AMG Package: $78,400 CAD

**Destination and Delivery Charges:**

* $2,250 CAD (estimated)

**Taxes and Fees:**

* HST (Harmonized Sales Tax) or GST (Goods and Services Tax) and PST (Provincial Sales Tax) may apply, depending on the province or territory of purchase.

**Total Price:**

* The total price of the GLC 300 Coupe in Canada can range from around $65,650 CAD to over $80,000 CAD, depending on the trim level, options, and taxes.

Keep in mind that prices may vary depending on the dealership, loca

In [4]:
import os
from langchain_community.document_loaders import WebBaseLoader

# set a real User-Agent so the site recognizes the request
os.environ["USER_AGENT"] = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
)

# Loads the Magnet Forensics careers page
loader = WebBaseLoader("https://jobs.lever.co/magnetforensics/6dcb96d2-e33d-4f82-8afa-3f92b9a7e235")
page_data = loader.load().pop().page_content

# Print the extracted text
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Magnet Forensics - Senior Software Engineer – Nexus TeamSenior Software Engineer – Nexus TeamCanadaResearch and Development – Engineering /Full-Time /RemoteApply for this jobWho We Are; What We Do; Where We’re GoingMagnet Forensics is a global leader in the development of digital investigative software that acquires, analyzes, and shares evidence from computers, smartphones, tablets, and IoT-related devices. We are continually innovating so our customers can deploy advanced and effective tools to protect their companies, communities, and countries. Serving thousands of customers globally, our solutions are playing a crucial role in modernizing digital investigations, helping investigators fight crime, protect assets, and guard national security. With employees based around the world, Magnet Forensics has been expanding our global presence. As a part of Magnet Forensics, you can expect to make a difference in the world, no matter what role you play. You’ll be supported through learning 

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
# prompt_extract | llm  this is piping 
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
print(res.content)

```json
{
  "role": "Senior Software Engineer – Nexus Team",
  "experience": "5+ years of relevant professional experience in software development",
  "skills": [
    "object-oriented programming (C# and ASP.NET Core)",
    "cloud platforms (AWS)",
    "containers (Docker)",
    "scaling SaaS products",
    "strong collaboration skills",
    "excellent communications skills"
  ],
  "description": "Design and build systems that are secure, reliable and maintainable. Lead technical design and architecture discussions, evaluating different approaches and determining the most effective solution. Break down complex problems into smaller problems to deliver value to our users incrementally. Collaborate with product managers and designers to understand customer needs and translate them into technical solutions. Own the development and maintenance of features, making trade-offs that balance quality, speed and scope while keeping long-term maintainability in mind. Foster a culture of continuous

In [7]:
# now we will convert it into a json which will make it a dictionary object
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer – Nexus Team',
 'experience': '5+ years of relevant professional experience in software development',
 'skills': ['object-oriented programming (C# and ASP.NET Core)',
  'cloud platforms (AWS)',
  'containers (Docker)',
  'scaling SaaS products',
  'strong collaboration skills',
  'excellent communications skills'],
 'description': 'Design and build systems that are secure, reliable and maintainable. Lead technical design and architecture discussions, evaluating different approaches and determining the most effective solution. Break down complex problems into smaller problems to deliver value to our users incrementally. Collaborate with product managers and designers to understand customer needs and translate them into technical solutions. Own the development and maintenance of features, making trade-offs that balance quality, speed and scope while keeping long-term maintainability in mind. Foster a culture of continuous improvement by always being cur

In [8]:
type(json_res)

dict

In [9]:
# now we will prepare chroma db
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"LangChain, Google Gemini, Streamlit, FAISS, In...",https://github.com/Kunal7575/KunalAssistant-En...
1,"LangChain, Google Gemini, Streamlit",https://www.linkedin.com/feed/update/urn:li:ac...
2,"TensorFlow, TensorFlow Lite, Python, Drone Tel...",https://stalwart-quokka-6ee6d2.netlify.app/
3,"React.js, Material-UI, YouTube API, Axios",https://64e4dd92d817ab20e7ff524a--superb-strud...
4,"Microsoft Excel, Data Analysis, Dashboard Design",https://1drv.ms/x/c/10bf769274e6b1e0/EZSgD3YYC...
5,"VHDL, Verilog, FPGA Design, Real-Time Embedded...",https://github.com/Kunal7575/Intruder-Detector
6,"Flask, Scikit-learn, GridSearchCV, K-Fold Vali...",https://legendary-melomakarona-3f8ca2.netlify....


In [12]:
# now we will start putting these one by one in chroma db 
# PersistentClient-> will create on a disk 
# client will create on a memory like chroma db on a memory
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
links = collection.query(query_texts=["Python, LangChain"], n_results=4).get('metadatas', [])
links

[[{'links': 'https://www.linkedin.com/feed/update/urn:li:activity:7355596818073698305/'},
  {'links': 'https://github.com/Kunal7575/KunalAssistant-End-to-End-GenAI-Project/'},
  {'links': 'https://legendary-melomakarona-3f8ca2.netlify.app/'},
  {'links': 'https://stalwart-quokka-6ee6d2.netlify.app/'}]]

In [20]:
job = json_res
job['skills']

['object-oriented programming (C# and ASP.NET Core)',
 'cloud platforms (AWS)',
 'containers (Docker)',
 'scaling SaaS products',
 'strong collaboration skills',
 'excellent communications skills']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Kunal Sharma, a Computer Engineering student specializing in Machine Learning with hands-on experience in AI, automation, IT infrastructure, and data analytics.  
        You are currently working as a Registrarial Systems & Technologies Co-op at the University of Guelph, supporting both on-site and remote users across six departments including Admissions, Enrolment Services, and Scheduling. Your role involves maintaining over 60 devices, patching systems, assisting with database-backed applications, supporting PHP/Drupal-based web systems, and documenting internal technical workflows. This position builds on your existing expertise in infrastructure automation, scripting, and data workflows, while also expanding your experience in OS-level troubleshooting, SQL and Excel-based data operations, and backend system maintenance.  
        
        Previously, you worked as a Deployment Analyst at The Co-operators, a Data Analyst at ThriftyAI, and have built advanced AI-powered projects such as an AI personal assistant trained on your resume, an AI-powered crop disease detection drone, and a housing price prediction web app. Your portfolio includes solutions in AI, machine learning, automation, data analysis, full-stack development, and real-time embedded systems.  
        
        When reaching out to potential clients, you should reference your **portfolio website** (https://66d46dbc12f6c2007a3b1dbc--rad-longma-0f1c7f.netlify.app/) as it showcases all your key projects and technical capabilities in one place.  
        
        Your goal is to write a tailored cold email to the client regarding the job mentioned above, showcasing your capabilities and past projects that align with their needs.  
        Also add the most relevant ones from the following links to demonstrate your portfolio: {link_list}  
        Do not provide a preamble.
        
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Senior Software Engineer – Nexus Team

Dear Hiring Manager,

I'm excited to apply for the Senior Software Engineer – Nexus Team position, and I'd like to highlight how my skills and experience align with your requirements. With a strong foundation in software development, cloud platforms, and a passion for innovative technologies, I'm confident I can make a valuable contribution to your team.

As a Computer Engineering student specializing in Machine Learning, I've developed a solid background in object-oriented programming, cloud platforms, and data analysis. My experience with AI, automation, IT infrastructure, and data analytics has equipped me with the skills to design and build secure, reliable, and maintainable systems. I'm particularly drawn to this role because of the emphasis on cloud platforms (AWS), containers (Docker), and scaling SaaS products, which aligns with my experience in infrastructure automation and scripting.

In my current role as Regist